In [1]:
#Pobieranie potrzebnych bibliotek
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
#Ładowanie danych i sprawdzenie kolumn
data = pd.read_excel('./artifacts/Mincer.xlsx')
print(data.columns)

data['dosw_sqrt'] = data['doswiadczenie']**2

data = data.replace({True: 1, False: 0})
print(data.head(1))

Index(['Unnamed: 0', 'doswiadczenie', 'pensja', 'najwyzsze', 'matura',
       'doswiadczenie_log', 'czy_ma_mature', 'czy_ma_lic', 'czy_ma_inz',
       'czy_ma_mgr', 'czy_ma_mgrinz', 'czy_ma_podyplomowe', 'czy_ma_mba',
       'czy_ma_phd', 'kujawsko_pomorskie', 'lubelskie', 'lubuskie', 'lodzkie',
       'malopolskie', 'mazowieckie', 'opolskie', 'podkarpackie', 'podlaskie',
       'pomorskie', 'slaskie', 'swietokrzyskie', 'warminsko_mazurskie',
       'wielkopolskie', 'zachodniopomorskie', 'male_miasto', 'srednie_miasto',
       'duze_miasto', 'kobieta', 'ln_wyn', 'edukacja_stp'],
      dtype='object')
   Unnamed: 0  doswiadczenie  pensja  najwyzsze  matura  doswiadczenie_log  \
0           0             39    2500        NaN       0           3.688879   

   czy_ma_mature  czy_ma_lic  czy_ma_inz  czy_ma_mgr  ...  \
0              0           0           0           0  ...   

   warminsko_mazurskie  wielkopolskie  zachodniopomorskie  male_miasto  \
0                    0              0 

In [3]:
# Deklarowanie zmiennych potrzebnych do stworzenia modelu
zmienne_niezalezne = [
    'doswiadczenie',
    'dosw_sqrt', 
    'kobieta', 
    'czy_ma_mature', 
    'czy_ma_lic', 
    'czy_ma_inz',
    'czy_ma_mgr', 
    'czy_ma_mgrinz', 
    'czy_ma_podyplomowe', 
    'czy_ma_mba',
    'czy_ma_phd'
]

y = np.log(data['pensja'])

kwantyle = [0.25, 0.50, 0.75]

wyniki = []

In [4]:
# Model
for q in kwantyle:
    formula = f'y ~ ' + ' + '.join(zmienne_niezalezne)
    model = smf.quantreg(formula, data=data)
    quantreg_results = model.fit(q=q)
    wyniki.append(quantreg_results)

print(wyniki)

[<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x0000024CA25CEF10>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x0000024CD906BC50>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x0000024CD905AE10>]


In [5]:
# Zapisanie wyników
for i, result in enumerate(wyniki):
    with open(f'./wyniki_model4_minc_kwant/eduakcja_zakodowana/z_dosw_sqrt/kwantyl_{kwantyle[i]}_wyniki.txt', 'w') as file:
        file.write(result.summary().as_text())

In [6]:
# Wyświetlenie estymatorów do interpretacji
decomposition_results = {}

for i, result in enumerate(wyniki):
    quantile = kwantyle[i] 
    decomposition_results[quantile] = {}
    
    for var in zmienne_niezalezne:
        effect = result.params[var]
        decomposition_results[quantile][var] = effect

for quantile, effects in decomposition_results.items():
    print(f"Quantile {quantile}:")
    for var, effect in effects.items():
        print(f"{var}: {effect:.4f}")

Quantile 0.25:
doswiadczenie: 0.0190
dosw_sqrt: -0.0004
kobieta: -0.3008
czy_ma_mature: 0.1676
czy_ma_lic: 0.2618
czy_ma_inz: 0.2921
czy_ma_mgr: 0.3170
czy_ma_mgrinz: 0.4260
czy_ma_podyplomowe: 0.3895
czy_ma_mba: 1.5964
czy_ma_phd: 0.4530
Quantile 0.5:
doswiadczenie: 0.0145
dosw_sqrt: -0.0003
kobieta: -0.3224
czy_ma_mature: 0.1880
czy_ma_lic: 0.2460
czy_ma_inz: 0.4378
czy_ma_mgr: 0.3799
czy_ma_mgrinz: 0.6088
czy_ma_podyplomowe: 0.4235
czy_ma_mba: 1.5895
czy_ma_phd: 0.5297
Quantile 0.75:
doswiadczenie: 0.0158
dosw_sqrt: -0.0003
kobieta: -0.3912
czy_ma_mature: 0.1966
czy_ma_lic: 0.2613
czy_ma_inz: 0.5555
czy_ma_mgr: 0.4659
czy_ma_mgrinz: 0.6878
czy_ma_podyplomowe: 0.4345
czy_ma_mba: 1.4832
czy_ma_phd: 0.4927


In [8]:
# VIF
new_df = data[zmienne_niezalezne].copy()

vif = pd.DataFrame()
vif["Variable"] = new_df.columns
vif["VIF"] = [variance_inflation_factor(new_df.values, i) for i in range(new_df.shape[1])]
print(vif)

              Variable        VIF
0        doswiadczenie  21.276635
1            dosw_sqrt  16.369530
2              kobieta   2.092762
3        czy_ma_mature   1.490451
4           czy_ma_lic   1.157053
5           czy_ma_inz   1.042236
6           czy_ma_mgr   1.534982
7        czy_ma_mgrinz   1.111825
8   czy_ma_podyplomowe   1.186927
9           czy_ma_mba   1.007681
10          czy_ma_phd   1.002963
